In [2]:
import pandas as pd

**MAIN**

In [38]:


#  **Get all words from excel (data frame)**
excel = pd.read_excel('Mon_words.xlsx')
data = pd.DataFrame(excel, columns = ['word_id', 'lemma', 'POS', 'Category', 'Derivation', 'Source word', 'Derived POS', 'Source POS', 'Inflection'])

# get noun words  
words = getAllWord(data, 'noun')
print(words)

#get verb
# verbs = getAllWord(data, 'verb', False)

#export noun words
# df = pd.DataFrame(words)
# df.to_csv('result.tsv', sep = '\t', index=False, encoding='utf-8')

#export verbs
# df = pd.DataFrame(verbs, columns = ['word_id', 'lemma', 'Inflection'])
# df.to_csv('verb-data.tsv', sep = '\t', index=False, encoding='utf-8')

[{'word_id': 1944401, 'lemma': 'нэр', 'POS': 'noun; verb', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan, 'Inflection': 'нэрийн, нэрд, нэрийг, нэрээс, нэрээр, нэртэй, нэр лүү, нэрээ'}, {'word_id': 1944402, 'lemma': 'отряд', 'POS': 'noun', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan, 'Inflection': 'отрядын, отрядод, отрядыг, отрядоос, отрядоор, отрядтой, отряд руу, отрядоо'}, {'word_id': 1944405, 'lemma': 'цутгах', 'POS': 'noun; verb', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan, 'Inflection': ''}, {'word_id': 1944406, 'lemma': 'хуйлрах', 'POS': 'verb; noun', 'Category': 'yes', 'Derivation': '-рах', 'Source word': 'хуйлах', 'Derived POS': 'verb', 'Source POS': 'verb', 'Inflection': ''}, {'word_id': 1944407, 'lemma': 'халих', 'POS': 'verb; noun', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan

# Test to read from file 

In [ ]:
excel = pd.read_excel('Mon_words.xlsx')
data = pd.DataFrame(excel, columns = ['word_id', 'lemma', 'POS', 'Category', 'Derivation', 'Source word', 'Derived POS', 'Source POS', 'Inflection'])

# print(data)
for index, row in data.iterrows():
  if index < 10: print(row['word_id'], row['lemma'], row['POS'], row['Category'], row['Derivation'], row['Source word'], row['Derived POS'], row['Source POS'], row['Inflection'])

In [19]:
for w in words: print(w)

{'word_id': 1944401, 'lemma': 'нэр', 'POS': 'noun; verb', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan, 'Inflection': 'нэрийн, нэрд, нэрийг, нэрээс, нэрээр, нэртэй, нэр лүү, нэрээ'}
{'word_id': 1944402, 'lemma': 'отряд', 'POS': 'noun', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan, 'Inflection': 'отрядын, отрядод, отрядыг, отрядоос, отрядоор, отрядтой, отряд руу, отрядоо'}
{'word_id': 1944405, 'lemma': 'цутгах', 'POS': 'noun; verb', 'Category': 'no', 'Derivation': nan, 'Source word': nan, 'Derived POS': nan, 'Source POS': nan, 'Inflection': ''}
{'word_id': 1944406, 'lemma': 'хуйлрах', 'POS': 'verb; noun', 'Category': 'yes', 'Derivation': '-рах', 'Source word': 'хуйлах', 'Derived POS': 'verb', 'Source POS': 'verb', 'Inflection': ''}


**Generate result data**

In [35]:
def getAllWord(data, pos, needOtherWords = True):
  words = []
  for index, row in data.iterrows():
    if pos in row['POS']:
      getNounWord(row, words, index, pos)
    elif needOtherWords:
      getWord(row, words)  
    #test 
    if index == 5:
      return words   
  # return words


def getNounWord(row, words, index, pos):
  wordList = sendRequest(row['lemma'], pos == 'verb') 
  if wordList:
    row['Inflection'] = get(wordList)
    getWord(row, words, True)
  else:
    getWord(row, words, True)

def getWord(row, words, state=False):
  inf = row['Inflection'] if row['Inflection'] == row['Inflection'] else ''
  w = { 'word_id': row['word_id'], 'lemma': row['lemma'], 'POS': row['POS'], 'Category': row['Category'], 'Derivation': row['Derivation'], 
        'Source word': row['Source word'], 'Derived POS': row['Derived POS'], 'Source POS': row['Source POS'], 'Inflection': inf }
  words.append(w)   

def get(wordList: list()):
  string = ''
  length = len(wordList)
  for w in wordList:
    if wordList.index(w) == length - 1:
      string = string + w
    else:
      string = string + w + ', '
  return string

**Send request to external service**

In [37]:
from bs4 import BeautifulSoup
from ordered_set import OrderedSet
import requests
import urllib

def sendRequest(word, getVerb = False):
  returnWords = OrderedSet()
  encoded = urllib.parse.quote(word)
  key = urllib.parse.quote('ТОЛЬ')
  url = 'http://toli.gov.mn/search?q=' + encoded + '&t=' + key
  soup = getFormattedHtml(url)

  match = soup.findAll('div', class_ = 'word-list-item')
  for row in match :
    pos = row.find('span', class_ = 'word-pos')
    if getVerb and '[үй.ү]' in pos.text:
      getWordList(row, returnWords, 'verb')
    elif getVerb == False and ('[ж.н]' in pos.text or '[гад.]' in pos.text) :
      getWordList(row, returnWords)
  return list(returnWords)

def getWordList(row, returnWords, pos = 'noun'):
  link = row.find('a')['href']
  soup = getFormattedHtml(link)
  if pos == 'noun':
    wordList = soup.findAll('span', class_ = 'word-inflected')
    for resWord in wordList:
      returnWords.add(resWord.text)
  elif pos == 'verb':
    wordList = soup.findAll('div', class_ = 'word-meaning')[1]
    returnWords.add(wordList.text)

def getFormattedHtml(url):
  res = getRequests(url)
  return BeautifulSoup(res, 'lxml')

def getRequests(url):
  res = requests.get(url)
  if res != None : return res.text

In [14]:
!pip install ordered_set

  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8210 sha256=738a177a891796986ce915737712c167d172eef6050ba6df3f95f7d060a027d8
  Stored in directory: /root/.cache/pip/wheels/e1/c6/9b/651d8a21d59b51a75ab9c070838f9231b8126421bc0569af47
Successfully built ordered-set
